## Objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - with or without balances at swap time (web3py fetch balance at block)
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    **Deliverable**
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

**Secondary objective: identify which group has the most profitable activity**
- Triggered Average of price by swap in/out by group of addresses

- Predict the return of the next candle based on the "history" of activity of the groups of addresses
    - Feature extracion
        - 

### Getting pairs with the highest liquidity

In [1]:
from py0xcluster.utils import web3_utils, query_utils
from py0xcluster.main_classes.classes import *

In [2]:
uni2_pools = Pools('univ2')
pools_summary, full_results = uni2_pools.get_pools(
    min_daily_volume = 100000,
    min_daily_txns = 200,
    min_txns_per_token = 50000,
    start_date = (2022,6,10),
    end_date = (2022,6,20)
    )
pools_summary.head(50)

skip varriable: 500
skip varriable: 1000


,pairAddress,dailyTxns,dailyVolumeUSD,reserveUSD,token0.totalLiquidity,token1.totalLiquidity,token0.txCount,token1.txCount,token0.symbol,token1.symbol,token0.id,token1.id
0,0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5,207.000000,4.458432e+06,1.561330e+08,3.316936e+07,1.041312e+08,2550131.0,7858389.0,DAI,USDC,0x6b175474e89094c44da98b954eedeac495271d0f,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
1,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,3840.600000,6.265653e+07,1.179343e+08,1.041312e+08,3.074635e+05,7858389.0,84650762.0,USDC,WETH,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,2241.900000,2.546473e+07,5.116743e+07,3.074635e+05,5.498092e+07,84650762.0,7263771.0,WETH,USDT,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdac17f958d2ee523a2206206994597c13d831ec7
3,0x3041cbd36888becc7bbcbc0045e3b1f144466f5f,287.600000,1.132654e+06,2.439658e+07,1.041312e+08,5.498092e+07,7858389.0,7263771.0,USDC,USDT,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xdac17f958d2ee523a2206206994597c13d831ec7
4,0x9c4fe5ffd9a9fc5678cfbd93aa2d4fd684b67c4c,308.000000,4.558700e+06,1.753999e+07,6.820157e+03,3.074635e+05,58234.0,84650762.0,PAXG,WETH,0x45804880de22913dafe09f4980848ece6ecbaf78,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
5,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,776.500000,6.396268e+06,1.413558e+07,3.316936e+07,3.074635e+05,2550131.0,84650762.0,DAI,WETH,0x6b175474e89094c44da98b954eedeac495271d0f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
6,0xd3d2e2692501a5c9ca623199d38826e513033a17,259.000000,2.779456e+06,1.403528e+07,1.506924e+06,3.074635e+05,846142.0,84650762.0,UNI,WETH,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
7,0xbb2b8038a1640196fbe3e38816f3e67cba72d940,296.500000,1.994026e+06,1.294543e+07,3.052725e+02,3.074635e+05,771521.0,84650762.0,WBTC,WETH,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
8,0x7a809081f991ecfe0ab2727c7e90d2ad7c2e411e,389.222222,3.250483e+06,1.068878e+07,1.041312e+08,7.307313e+13,7858389.0,74690.0,USDC,CAW,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xf3b9569f82b18aef890de263b84189bd33ebe452
9,0x7b73644935b8e68019ac6356c40661e1bc315860,309.428571,1.478200e+06,9.382670e+06,1.690648e+13,3.074635e+05,420902.0,84650762.0,ELON,WETH,0x761d38e5ddf6ccf6cf7c55759d5210750b5d60f3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


In [3]:
uni2_trades = TradesDEX('X')
uni2_pools.pair_addresses

all_swaps = uni2_trades.get_swaps(
    pair_addresses = uni2_pools.pair_addresses[:],
    min_amoutUSD = 10000,
    start_date = (2022,6,10),
    end_date = (2022,6,20),
    pairs_batch_size = 3
)

skip varriable: 500
skip varriable: 1000
skip varriable: 1500
skip varriable: 2000
skip varriable: 2500
skip varriable: 3000
skip varriable: 3500
skip varriable: 4000
skip varriable: 4500
skip varriable: 5000
skip varriable: 5500
skip varriable: 6000
request aborted
{'errors': [{'message': 'The `skip` argument must be between 0 and 5000, but is 5500'}]}
swaps collected so far: 5500
skip varriable: 500
skip varriable: 1000
skip varriable: 1500
skip varriable: 2000
skip varriable: 2500
skip varriable: 3000
skip varriable: 3500
skip varriable: 4000
swaps collected so far: 8712
skip varriable: 500
skip varriable: 1000
skip varriable: 1500
skip varriable: 2000
skip varriable: 2500
skip varriable: 3000
swaps collected so far: 10731
skip varriable: 500
skip varriable: 1000
skip varriable: 1500
swaps collected so far: 11414
skip varriable: 500
skip varriable: 1000
skip varriable: 1500
swaps collected so far: 12168
skip varriable: 500
skip varriable: 1000
swaps collected so far: 12425
skip varr

### 

In [4]:
etherscan_API_path = r'C:\Users\phar0732\Documents\ID\etherscan\etherscan_api.csv'
etherscan_token = str(pd.read_csv(etherscan_API_path, header=None).values[0][0])
etherscan_token

address0 = '0x424a31f5caa3155437ebed0ce80169639f6689d5'
address1 = '0x728BD56A965dC3B9aDA745301C2f731599d2fe13'


In [5]:
web3_utils.get_nonce('0x424a31f5caa3155437ebed0ce80169639f6689d5')

1